# SAM / ASAM Experiment Notebook

Notebook ini dipakai untuk menjalankan eksperimen:

- Dataset: **CIFAR-10** dan **SVHN**
- Model: **ResNet-18 (versi 32×32)** dan **WideResNet**
- Optimizer:
  - SGD
  - AdamW
  - SAM(SGD)
  - SAM(AdamW)

Notebook ini akan:
- Memanggil `train.py` dengan kombinasi argumen yang berbeda.
- Memudahkan tes cepat (epochs kecil) maupun eksperimen penuh.

In [1]:
import os
import sys
import subprocess

import torch

print("Python version :", sys.version)
print("Torch version  :", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device   :", torch.cuda.get_device_name(0))

Python version : 3.10.19 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 16:41:31) [MSC v.1929 64 bit (AMD64)]
Torch version  : 2.5.1+cu121
CUDA available: True
CUDA device   : NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [2]:
print("Current working dir:", os.getcwd())
assert os.path.exists("train.py"), "train.py tidak ditemukan di folder ini. Pastikan kamu ada di folder example/"


Current working dir: c:\Users\Aufii\Documents\Informatika\SEM 5\DL\PROJECT_AKHIR\sam-pytorch\projectakhir-dl-sam-pytorch\example


Fungsi helper

In [3]:
import subprocess

def run_once(dataset, model, optimizer, seed, epochs=20, lr=0.1, wd=5e-4):
    cmd = [
        "python", "train.py",
        "--dataset", dataset,
        "--model", model,
        "--optimizer", optimizer,
        "--epochs", str(epochs),
        "--learning_rate", str(lr),
        "--weight_decay", str(wd),
        "--seed", str(seed)
    ]
    
    print("=====================================")
    print("Running:", " ".join(cmd))
    print("=====================================")

    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)


## Sanity Check: 1 Run Cepat (CIFAR-10 + ResNet18 + SGD)

In [6]:
!python train.py --dataset cifar10 --model resnet18 --optimizer sgd --epochs 1 --learning_rate 0.1 --weight_decay 5e-4

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨

┃           0  ┃      4.9670  │     10.59 %  ┃   1.000e-01  │   00:20 min  ┠┈▒┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┨
┃           0  ┃      4.0044  │     11.41 %  ┃   1.000e-01  │   00:24 min  ┠┈█░┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┨
┃           0  ┃      2.0399  │     10.39 %  ┃   1.000e-01  │   00:29 min  ┠┈██┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┨
┃           0  ┃      1.8523  │     10.23 %  ┃   1.000e-01  │   00:34 min  ┠┈██▓┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┨
┃           0  ┃      1.7808 

In [8]:
import sys
!{sys.executable} train.py --dataset cifar10 --model resnet18 --optimizer sgd --epochs 1 --learning_rate 0.1 --weight_decay 5e-4

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨

┃           0  ┃      4.1226  │     11.46 %  ┃   1.000e-01  │   05:16 min  ┠┈▒┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┨
┃           0  ┃      2.6055  │     10.23 %  ┃   1.000e-01  │   05:22 min  ┠┈█░┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┨
┃           0  ┃      1.9842  │     10.94 %  ┃   1.000e-01  │   05:27 min  ┠┈██┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┨
┃           0  ┃      1.8363  │      9.22 %  ┃   1.000e-01  │   05:32 min  ┠┈██▓┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┨
┃           0  ┃      1.7571 

## SGD

In [4]:
datasets = ["cifar10", "svhn"]
model = "resnet18"
seeds = [0, 1, 2]

In [ ]:
for dataset in datasets:
    for seed in seeds:
        run_once(dataset, model, "sgd", seed)

## AdamW

In [ ]:
for dataset in datasets:
    for seed in seeds:
        run_once(dataset, model, "adamw", seed)

## SGD (SAM)

In [ ]:
for dataset in datasets:
    for seed in seeds:
        run_once(dataset, model, "sam_sgd", seed)

## AdamW (SAM)

In [ ]:
for dataset in datasets:
    for seed in seeds:
        run_once(dataset, model, "sam_adamw", seed)